In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from tqdm.notebook import tqdm
import matplotlib as plt

import sys
sys.path.append('..')

from src.datasets import load_dataset
from src.evaluation.evaluation import downvote_seen_items, topn_recommendations, model_evaluate

import json
import torch
from src.models import NGCF, ALS, eALS, iALS

In [ ]:
data_path = '../data'

training_yelp, testset_yelp, holdout_yelp, training_matrix_yelp, data_description_yelp = load_dataset('yelp', data_path)
training_movielens, testset_movielens, holdout_movielens, training_matrix_movielens, data_description_movielens = load_dataset('movielens', data_path)

In [ ]:
with open('../configs/ngcf_base.json') as f:
    config = json.load(f)

n_users, n_items = training_matrix_movielens.shape
config['model_args']['n_users'] = n_users
config['model_args']['n_items'] = n_items

model = NGCF(**config['model_args'])
model.load_state_dict(torch.load('../lightning_logs/version_30/checkpoints/epoch=399-step=2399.ckpt')['state_dict'])
model.eval();

In [54]:
# !!! scores_X - результат работы модели X, т.е. numpy.ndarray шейпа: scores_X.shape=(число юзеров в холдауте, число всех айтемов),
# где элементы - предсказанные скоры айтемов для каждого юзера (не индексы айтемов, а именно скоры).
scores_yelp = model.score_users(holdout_yelp.userid.values)
# scores_movielens = model.score_users(holdout_movielens.userid.values)

In [55]:
downvote_seen_items(scores_yelp, testset_yelp, data_description_yelp)
# downvote_seen_items(scores_movielens, testset_movielens, data_description_movielens)

In [56]:
topn = 10

recs_yelp = topn_recommendations(scores_yelp, topn=topn)
# recs_movielens = topn_recommendations(scores_movielens, topn=topn)

In [1]:
# import implicit
# als = implicit.als.AlternatingLeastSquares()
# als.fit(training_matrix_movielens)
# recs_movielens = als.recommend_all(training_matrix_movielens, filter_already_liked_items=True, N=topn)

In [58]:
datasets = ['Yelp: ', 'MovieLens: ']

In [59]:
hr, mrr, ndcg, cov = model_evaluate(recs_yelp, holdout_yelp, data_description_yelp, topn=topn)
print(datasets[0])
print(f'HR@{topn} = {hr:.4f}')
print(f'MRR@{topn} = {mrr:.4f}')
print(f'nDCG@{topn} = {ndcg:.4f}')
print(f'COV@{topn} = {cov:.4f}')

Yelp: 
HR@10 = 0.0418
MRR@10 = 0.0155
nDCG@10 = 0.0216
COV@10 = 0.0625


In [45]:
hr, mrr, ndcg, cov = model_evaluate(recs_movielens, holdout_movielens, data_description_movielens, topn=topn) 
print(datasets[1])
print(f'HR@{topn} = {hr:.4f}')
print(f'MRR@{topn} = {mrr:.4f}')
print(f'nDCG@{topn} = {ndcg:.4f}')
print(f'COV@{topn} = {cov:.4f}')

MovieLens: 
HR@10 = 0.0990
MRR@10 = 0.0359
nDCG@10 = 0.0505
COV@10 = 0.3653
